In [1]:
import requests
import csv
import time
from datetime import datetime

api_url = 'https://earthquake.usgs.gov/fdsnws/event/1/'
api_method = 'query?'
api_method_2 = 'count?'
api_format = 'format=geojson'
api_starttime = '1918-08-21'
# api_endtime = '2018-08-21'
api_minlatitude = '21.821'
api_minlongitude = '119.971'
api_maxlatitude = '25.384'
api_maxlongitude = '122.014'

In [2]:
url ='{0}{1}{2}&starttime={3}&minlatitude={4}&maxlatitude={5}&minlongitude={6}&maxlongitude={7}'.format(api_url,api_method,api_format,api_starttime,api_minlatitude,api_maxlatitude,api_minlongitude,api_maxlongitude)
url_2 = '{0}{1}{2}&starttime={3}&minlatitude={4}&maxlatitude={5}&minlongitude={6}&maxlongitude={7}'.format(api_url,api_method_2,api_format,api_starttime,api_minlatitude,api_maxlatitude,api_minlongitude,api_maxlongitude)

In [3]:
def timestamp_to_utc_strtime(timestamp):
        """将 13 位整数的毫秒时间戳转化成 utc 时间 (字符串格式，含毫秒)
        :param timestamp: 13 位整数的毫秒时间戳 (1456402864242)
        :return: 返回字符串格式 {str}'2016-02-25 12:21:04.242000'
        """
        utc_str_time = datetime.utcfromtimestamp(timestamp / 1000.0).strftime('%Y-%m-%d %H:%M:%S.%f')
        return utc_str_time

def request():
    response = requests.get(url)
    response_2 = requests.get(url_2)

    data = response.json()
    count = response_2.json()
    
    place = []
    mag = []
    time = []
    
    for i in range(0,len(data['features'])):
        mag.append(data['features'][i]['properties']['mag'])
        place.append(data['features'][i]['properties']['place'])
        time.append(data['features'][i]['properties']['time'])
    #从json格式中提取所需信息
    
    utc_time = []
    for timestr in time:
        timestamp_to_utc_strtime(timestr)
        utc_time.append(timestamp_to_utc_strtime(timestr)[:19])

    return (utc_time,mag,place)

lasttime = ''
while True:
    utc_time,mag,place = request()
    if utc_time[0] > lasttime:
        print('A %d magnitude earthquake occurred on %s, at %s, Latest earthquakes %s'%(mag[0],place[0],utc_time[0],'https://earthquake.usgs.gov/earthquakes/eventpage/ci38252496#executive'))

        with open('taiwan_earthquake_0822.csv','w',newline='') as f:
            writer = csv.writer(f)
            header = ['Place','Mag','Time']
            writer.writerow(header)
            for i in range(0,len(utc_time)):
                writer.writerow([place[i],mag[i],utc_time[i]])
            f.close() 
        lasttime = utc_time[0]
    else:
        continue

A 4 magnitude earthquake occurred on 4km NNE of Buli, Taiwan, at 2018-08-18 00:15:17, Latest earthquakes https://earthquake.usgs.gov/earthquakes/eventpage/ci38252496#executive


KeyboardInterrupt: 